# Primary Keys and Foreign Keys in Snowflake

## Learning Objectives

By the end of this notebook, you will understand:
1. What Primary Keys and Foreign Keys are
2. How to create tables with Primary and Foreign Key constraints
3. How referential integrity prevents data inconsistencies
4. Why you cannot delete parent records when child records exist
5. How to properly delete data when foreign key relationships exist

## Table of Contents

1. [Introduction to Keys](#introduction)
2. [Creating Tables with Keys](#creating-tables)
3. [Inserting Sample Data](#inserting-data)
4. [Understanding Referential Integrity](#referential-integrity)
5. [Deletion Scenarios](#deletion-scenarios)
6. [Best Practices](#best-practices)


## 1. Introduction to Keys

### Primary Key (PK)
- **Uniquely identifies** each row in a table
- **Cannot be NULL**
- **Cannot have duplicate values**
- A table can have only **one primary key**

### Foreign Key (FK)
- A column (or set of columns) that **references** the primary key of another table
- Establishes a **relationship** between two tables
- The table with the foreign key is called the **child table**
- The table being referenced is called the **parent table**
- Foreign keys enforce **referential integrity**

### Referential Integrity
- Ensures that relationships between tables remain consistent
- Prevents orphaned records (child records without a parent)
- In Snowflake, foreign keys are **informational** by default (not enforced), but we can still use them to maintain data quality


## 2. Creating Tables with Keys

We'll create two tables:
- **Customers** (Parent Table): Stores customer information
- **Orders** (Child Table): Stores order information, references Customers

Let's start by creating the parent table with a PRIMARY KEY:


In [ ]:
-- Step 1: Create the Customers table (Parent Table) with PRIMARY KEY
CREATE OR REPLACE TABLE customers (
    customer_id INT PRIMARY KEY,  -- Primary Key: uniquely identifies each customer
    customer_name VARCHAR(100) NOT NULL,
    email VARCHAR(100),
    city VARCHAR(50)
);

-- Verify the table was created
SELECT * FROM customers;


Now let's create the Orders table (Child Table) with a FOREIGN KEY that references the Customers table:


In [ ]:
-- Step 2: Create the Orders table (Child Table) with FOREIGN KEY
CREATE OR REPLACE TABLE orders (
    order_id INT PRIMARY KEY,  -- Primary Key for orders table
    customer_id INT NOT NULL,  -- Foreign Key: references customers.customer_id
    order_date DATE,
    order_amount DECIMAL(10, 2),
    
    -- Define the foreign key constraint
    CONSTRAINT fk_customer 
        FOREIGN KEY (customer_id) 
        REFERENCES customers(customer_id)
);

-- Verify the table was created
SELECT * FROM orders;


### Understanding the Relationship

```
Customers (Parent Table)          Orders (Child Table)
┌─────────────┐                   ┌─────────────┐
│ customer_id │◄──────────────────│ customer_id │ (Foreign Key)
│ (PK)        │                   │             │
│ name        │                   │ order_id    │ (PK)
│ email       │                   │ order_date  │
│ city        │                   │ amount      │
└─────────────┘                   └─────────────┘
    1                                  *
    │                                  │
    └─────────── One-to-Many ──────────┘
    
One customer can have many orders, but each order belongs to one customer.
```


## 3. Inserting Sample Data

Let's insert data into both tables. **Important**: We must insert into the parent table (Customers) first, then the child table (Orders).


In [ ]:
-- Step 3: Insert data into Customers table (Parent Table) FIRST
INSERT INTO customers (customer_id, customer_name, email, city) VALUES
    (1, 'John Smith', 'john.smith@email.com', 'New York'),
    (2, 'Jane Doe', 'jane.doe@email.com', 'Los Angeles'),
    (3, 'Bob Johnson', 'bob.johnson@email.com', 'Chicago'),
    (4, 'Alice Williams', 'alice.williams@email.com', 'Houston');

-- Verify the data
SELECT * FROM customers ORDER BY customer_id;


Now let's insert orders. Notice that each `customer_id` in the orders table must exist in the customers table:


In [ ]:
-- Step 4: Insert data into Orders table (Child Table)
-- Each customer_id must exist in the customers table
INSERT INTO orders (order_id, customer_id, order_date, order_amount) VALUES
    (101, 1, '2024-01-15', 150.00),  -- Order for John Smith (customer_id = 1)
    (102, 1, '2024-01-20', 75.50),   -- Another order for John Smith
    (103, 2, '2024-01-18', 200.00),  -- Order for Jane Doe (customer_id = 2)
    (104, 3, '2024-01-22', 99.99),   -- Order for Bob Johnson (customer_id = 3)
    (105, 2, '2024-01-25', 125.75);  -- Another order for Jane Doe

-- Verify the data
SELECT * FROM orders ORDER BY order_id;


In [ ]:
-- Step 5: View the relationship between customers and orders
SELECT 
    c.customer_id,
    c.customer_name,
    o.order_id,
    o.order_date,
    o.order_amount
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
ORDER BY c.customer_id, o.order_id;


## 4. Understanding Referential Integrity

### What happens if we try to insert an order with a non-existent customer_id?

Let's try to insert an order for a customer that doesn't exist:


In [ ]:
-- Step 6: Try to insert an order with a customer_id that doesn't exist
-- This should fail due to referential integrity constraint
-- Note: In Snowflake, foreign key constraints are informational by default
-- but this demonstrates the concept

INSERT INTO orders (order_id, customer_id, order_date, order_amount) 
VALUES (106, 999, '2024-01-30', 50.00);  -- customer_id 999 doesn't exist!

-- Check if it was inserted (it shouldn't be if constraints are enforced)
SELECT * FROM orders WHERE order_id = 106;


## 5. Deletion Scenarios - The Core Concept

### The Rule: **You cannot delete a parent record if child records exist**

This is the fundamental principle of referential integrity. Let's demonstrate this step by step:


### Scenario 1: Attempting to Delete a Customer with Existing Orders ❌

Let's try to delete a customer who has orders. This should fail or cause an error:


In [ ]:
-- Step 7: Try to delete John Smith (customer_id = 1) who has 2 orders
-- This should fail because there are child records (orders) referencing this customer

-- First, let's check how many orders John Smith has
SELECT 
    c.customer_id,
    c.customer_name,
    COUNT(o.order_id) as order_count
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE c.customer_id = 1
GROUP BY c.customer_id, c.customer_name;


In [ ]:
-- Now try to delete the customer
-- In Snowflake, this might succeed if foreign keys are not enforced,
-- but in databases with enforced constraints, this would fail
DELETE FROM customers WHERE customer_id = 1;

-- Check if the customer was deleted
SELECT * FROM customers WHERE customer_id = 1;

-- Check the orders - they would become "orphaned" if the customer was deleted
SELECT * FROM orders WHERE customer_id = 1;


**Important Note**: In Snowflake, foreign key constraints are **informational** by default (not enforced at runtime). However, the principle still applies in production databases with enforced constraints, and it's a best practice to follow this rule.

### Scenario 2: Proper Deletion Order ✅

To delete a customer with orders, you must:
1. **First**: Delete all child records (orders)
2. **Then**: Delete the parent record (customer)

Let's demonstrate the correct approach:


In [ ]:
-- Step 8: CORRECT WAY - Delete child records first, then parent

-- First, let's restore John Smith if he was deleted (for demonstration)
-- Note: Snowflake doesn't support ON CONFLICT, so we'll use MERGE or check first
MERGE INTO customers AS target
USING (SELECT 1 AS customer_id, 'John Smith' AS customer_name, 'john.smith@email.com' AS email, 'New York' AS city) AS source
ON target.customer_id = source.customer_id
WHEN NOT MATCHED THEN INSERT (customer_id, customer_name, email, city) 
VALUES (source.customer_id, source.customer_name, source.email, source.city);

-- Restore his orders if needed
MERGE INTO orders AS target
USING (SELECT 101 AS order_id, 1 AS customer_id, '2024-01-15'::DATE AS order_date, 150.00 AS order_amount) AS source
ON target.order_id = source.order_id
WHEN NOT MATCHED THEN INSERT (order_id, customer_id, order_date, order_amount) 
VALUES (source.order_id, source.customer_id, source.order_date, source.order_amount);

MERGE INTO orders AS target
USING (SELECT 102 AS order_id, 1 AS customer_id, '2024-01-20'::DATE AS order_date, 75.50 AS order_amount) AS source
ON target.order_id = source.order_id
WHEN NOT MATCHED THEN INSERT (order_id, customer_id, order_date, order_amount) 
VALUES (source.order_id, source.customer_id, source.order_date, source.order_amount);


In [ ]:
-- Step 8a: FIRST - Delete all orders for customer_id = 1
DELETE FROM orders WHERE customer_id = 1;

-- Verify the orders are deleted
SELECT * FROM orders WHERE customer_id = 1;


In [ ]:
-- Step 8b: THEN - Now we can safely delete the customer
DELETE FROM customers WHERE customer_id = 1;

-- Verify the customer is deleted
SELECT * FROM customers WHERE customer_id = 1;


### Scenario 3: Deleting a Customer with No Orders ✅

A customer with no orders can be deleted directly:


In [ ]:
-- Step 9: Delete a customer who has no orders
-- Alice Williams (customer_id = 4) has no orders, so we can delete her directly

-- First, verify she has no orders
SELECT 
    c.customer_id,
    c.customer_name,
    COUNT(o.order_id) as order_count
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE c.customer_id = 4
GROUP BY c.customer_id, c.customer_name;


In [ ]:
-- Since there are no orders, we can delete the customer directly
DELETE FROM customers WHERE customer_id = 4;

-- Verify
SELECT * FROM customers WHERE customer_id = 4;


### Scenario 4: Using CASCADE DELETE (Advanced)

Some databases support CASCADE DELETE, which automatically deletes child records when a parent is deleted. However, **use this with caution** as it can lead to unintended data loss.

In Snowflake, you would need to handle this manually or use stored procedures.

Let's see the current state of our tables:


In [ ]:
-- Step 10: View current state of both tables
SELECT 'Customers Table' as table_name;
SELECT * FROM customers ORDER BY customer_id;

SELECT 'Orders Table' as table_name;
SELECT * FROM orders ORDER BY order_id;


## 6. Best Practices and Summary

### Key Takeaways:

1. **Primary Key**: Uniquely identifies each row in a table
2. **Foreign Key**: Creates a relationship between tables, referencing a primary key
3. **Referential Integrity**: Ensures data consistency between related tables

### Deletion Rules:

✅ **CORRECT**: Delete child records first, then parent records
```sql
DELETE FROM orders WHERE customer_id = 1;      -- Delete child first
DELETE FROM customers WHERE customer_id = 1;   -- Then delete parent
```

❌ **INCORRECT**: Trying to delete parent when children exist
```sql
DELETE FROM customers WHERE customer_id = 1;   -- This will fail if orders exist
```

### Best Practices:

1. **Always check for child records** before deleting parent records
2. **Use transactions** to ensure atomicity (all or nothing)
3. **Consider soft deletes** (marking records as deleted) instead of hard deletes
4. **Document relationships** clearly in your database schema
5. **Use CASCADE DELETE carefully** - it can cause unintended data loss

### Practical Query to Check Before Deletion:

```sql
-- Before deleting a customer, check if they have orders
SELECT 
    c.customer_id,
    c.customer_name,
    COUNT(o.order_id) as order_count
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE c.customer_id = ?  -- Replace ? with the customer_id you want to delete
GROUP BY c.customer_id, c.customer_name;
```

If `order_count > 0`, you must delete the orders first!


### Exercise: Practice What You Learned

Try these exercises:

1. **Check relationships**: Write a query to find all customers and their order counts
2. **Safe deletion**: Delete customer_id = 2 (Jane Doe) and all her orders in the correct order
3. **Verify**: After deletion, verify that both the customer and her orders are gone
4. **Restore**: Re-insert the deleted data to restore the original state


In [ ]:
-- Exercise 1: Find all customers and their order counts
-- Write your query here:




In [ ]:
-- Exercise 2: Delete customer_id = 2 and all her orders
-- Step 1: Delete orders first




In [ ]:
-- Step 2: Delete the customer




In [ ]:
-- Exercise 3: Verify the deletion
-- Check if customer_id = 2 exists




In [ ]:
-- Check if any orders for customer_id = 2 exist




---

## Additional Notes for Snowflake

### Foreign Key Constraints in Snowflake:

- Snowflake supports **declarative referential integrity** (foreign key constraints)
- However, foreign key constraints are **informational** by default and not enforced at runtime
- This means Snowflake will validate the constraint definition but won't prevent invalid data insertion
- To enforce constraints, you may need to:
  - Use application-level validation
  - Implement stored procedures with validation logic
  - Use triggers (if supported in your Snowflake edition)

### Why This Still Matters:

Even though Snowflake doesn't enforce foreign keys by default, understanding these concepts is crucial because:
1. **Data Quality**: Maintaining referential integrity ensures data consistency
2. **Query Performance**: Proper relationships help with query optimization
3. **Business Logic**: Many business rules depend on maintaining these relationships
4. **Other Databases**: Most production databases (PostgreSQL, MySQL, SQL Server) enforce these constraints strictly

---

**End of Tutorial** 🎓
